# Turing-Maschinen-Simulation

von W. Conen, Version 1.0

Dieses kurze Notebook gibt die Möglichkeit, eine simple Implementierung einer Variante der von 
[Alan M. Turing](https://de.wikipedia.org/wiki/Alan_Turing) entwickelten Turing-Maschinen zu erproben. Über Turing wird an
anderem Orte, z.B. in der [Biografie von Andrew Hodges](http://www.turing.org.uk/), sehr eloquent und informativ berichtet. Seine Lebensgeschichte ist 
spannend, ergreifen und, leider, auch traurig. Seine Beiträge in der Entstehungphase der Informatik sind von grundlegender 
Bedeutung für das, was folgte.

## Elementares

Unsere STM (STM steht für Simple-Turing-Maschine) kann Folgendes:

Sie arbeitet auf einem endlosen Band aus nebeneinander liegenden Kästchen, in denen etwas stehen kann - oder auch nicht (dann denken wir uns dort ein Leerzeichen). Alles, was auf dem Band stehen darf, stammt aus dem sogenannten Bandalphabet, z.B. könnten dort Einsen (1) und Nullen (0) erlaubt sein - und das Leerzeichen,  klar. Oft trennt man übrigens zwischen Bandalphabet und Eingabealphabet (s. die Definition einer deterministischen Turingmaschine in https://de.wikipedia.org/wiki/Turingmaschine), aber, ehrlich gesagt, ist das noch etwas zu formal und wir können das hier ignorieren.

Die STM hat einen Schreib/Lese-Kopf, der auf genau einer Bandposition steht. Wenn sich eine Eingabe auf dem Band befindet (was man sehr oft annimmt), dann unterstellt man, dass der Kopf auf dem am weitestens links stehenden Zeichen der Eingabe steht. Die Eingabe ist endlich, es gibt also immer einen "leftmost character".

Die STM arbeitet dann ein Programm ab. Sie ist zu jedem Zeitpunkt in einem wohlbestimmten Zustand. Zu Beginn ist sie in einem ausgezeichneten Startzustand. Wenn Sie nicht bereits in einem Endzustand ist, dann liest sie das Zeichen unter ihrem Schreib/Lese-Kopf und schaut in ihrem Programm nach, was für das Paar (Zustand,Gelesenes_Zeichen) nun zu tun ist. Wenn sie in ihrem Programm keine passende Anweisung findet, bleibt Sie stehen. Im Grunde ist sie dann "kaputt". Wenn sie eine passende Anweisung findet, führt sie diese aus.

Wie sieht so eine Ausführung aus? Die Maschine schreibt zunächst ein Zeichen an die Position, an der sie steht (immer noch die gleiche Position, an der sie "Gelesenes_Zeichen" las...klar), bewegt sich dann nach links (L) oder rechts (R) (manche Maschinen dürfen auch stehen bleiben (-), unsere unten auch!) und geht dabei in einen Folgezustand über. Das war's schon. Dann schaut sie an der neuen Position (die manchmal auch die alte sein kann) wieder nach dem dort stehenden Zeichen usw.

Das Programm ist im übrigen auch endlich (es gibt also auch nur endlich viele Zustände), ebenso ist das (Band)alphabet endlich. 

Eine Berechnung, also eine technisch erfolgreiche Ausführung eines Programms, ergibt sich dann, wenn man in einen der ausgezeichneten Endzustände der Maschine gelangt (einen muss es immer mindestens geben). Dann bleibt die Maschine stehen und ist "fertig".

Ob das erzielte Ergebnis korrekt ist oder nicht, weiß man dann noch nicht. Eine korrekte Berechnung ist es, wenn die Berechnung auch noch korrekt ist ;)

Verrückt ist, dass man diese einfachen Maschine so bauen kann, das man genau das lösen kann, was man intuitiv für berechenbar hält (das ist die sogenannte Church-Turing-These).

Turing hat diese Maschinen auch in einer universellen Form vorgeschlagen, die ein Programm ausführen, das in der Lage ist, eine andere Turingmaschine (jede!) zu "simulieren", die als Programmcode auf dem Band steht und die dann auf eine Eingabe angewendet wird. Das ist natürlich noch viel cooler - und war ein wichtiges logisches Vorbild für speicherprogrammierbare Rechnerarchitekturen (ja ja, ungefähr all das, was wir heute so im Alltag verwenden...Notebooks, Handys, Tablets...).

## Mini-Turing-Maschinen-Simulator

Wir erläutern jetzt kurz die Nutzung des unten stehenden Programms:

Die Turingmaschine startet auf dem ersten linken Zeichen der Eingabe (wenn es
eine Eingabe gibt)- um das zu garantieren, stellen wir die Eingabe passend zur
Verfügung.

Sie befindet sich zu Beginn in einem Anfangszustand z_start (den Namen legen 
wir so fest). Die Abarbeitung ist beendet, wenn die Maschine den Zustand z_ende
erreicht (wir erlauben unten nur einen Endzustand).

Die Maschine kann Befehle im folgenden Format interpretieren:
<pre>
  (zustand_x,zeichen_1)->(zeichen_2,zustand_y,bewegung_b)
</pre>
Bedeutung: Wenn die Maschine im Zustand zustand_x das Zeichen zeichen_1
sieht, dann schreibt sie das Zeichen zeichen_2, geht in den Zustand
zustand_y und führt die Bewegung b aus. b kann L,R oder - sein (L für
links, R für rechts, - für "nicht bewegen"). Hierbei können zustand_x und
zustand_y auch die gleichen Zustände sein, ebenso zeichen_1 und zeichen_2.

Die Befehle werden als 5-elementige Listen dargestellt mit Strings als
Elemente:
<pre>
    ['z_start','0','1','z_start','R']
</pre>

Ein Programm ist eine Liste von Befehlen:
<pre>
  [
    ['z_start','0','1','z_start','R'],
    ['z_start','1','0','z_start','R'],
    ['z_start','_','_','z_ende','L']
  ]
</pre>

Hierbei steht _ für das Leerzeichen (oder "Leersymbol").

In [1]:
version = '1.0'

## This is a simple deterministic turing machine

class Turing(object):
    def __init__(self,programm,input=['_']):
        self.programm = programm
        self.input = input # Vorsicht...
        self.zustand = 'z_start'
        self.befehls_zeiger = 0
        self.input_position = 0
        self.end_position = len(input)-1

    def get_befehl(self,zeichen,zustand):
        # Geht sehr viel effizienter...aber hier der Einfachheit halber:
        for befehl in self.programm:
            if befehl[0] == zustand and befehl[1] == zeichen:
                return befehl
        return None

    def set_zustand(self,zustand):
        self.zustand = zustand

    def get_zustand(self):
        return self.zustand
        
    def read(self):
        return self.input[self.input_position]

    def write(self,char):
        self.input[self.input_position] = char

    def move(self,richtung):
        if richtung == 'L':
            if self.input_position == 0:
                # Hänge vorne an
                # Stattdessen könnten wir auch
                # einfach input.insert(0.'_') verwenden
                # und die end_position um eins hochsetzen
                # aber so wird ein wenig deutlicher,
                # das wir uns AUF einem unendlichen
                # Band voll mit Leerzeichen (_) bewegen
                mehr = ['_','_','_','_','_']
                l_mehr = len(mehr)
                self.input = mehr + self.input
                self.end_position += l_mehr
                self.input_position += l_mehr
            self.input_position -= 1
        if richtung == 'R':
            if self.input_position == self.end_position:
                # Hänge hinten noch etwas an
                mehr = ['_','_','_','_','_']
                l_mehr = len(mehr)
                self.input = self.input + mehr
                self.end_position += l_mehr
            self.input_position += 1
        if richtung == '-':
            pass # Tue nichts

    def show_input(self):
        return ''.join(self.input)
        
    def show_input_pointer(self):
        vorspann = ""
        for i in range(0,self.input_position):
            vorspann += ' '  # ineffizient, aber ok hier
        vorspann += "^"
        return vorspann
            
    def step(self,debug=False,count=0):
        zeichen_1 = self.read()
        zustand_x = self.get_zustand()
        befehl = self.get_befehl(zeichen_1,zustand_x)
        if not befehl:
            print("FEHLER: Maschine ist stehengeblieben!")
            return False
        if debug: print(count,": -- Execute: ",befehl)
        self.fuehre_aus(befehl)
        if debug: print("    --> ", self.show_input())
        if debug: print("    --> ", self.show_input_pointer())
        zustand_y = self.get_zustand()
        if zustand_y == 'z_ende':
            print("ERFOLG: Die Maschine hat den Endzustand erreicht!")
            return True
        return self.step(debug,count+1)   
        
    def fuehre_aus(self,befehl):
        self.write(befehl[2])
        self.set_zustand(befehl[3])
        self.move(befehl[4])
        
## Ende des Simulators

## Invertieren von 0 und 1, es wird also aus jeder 0 eine 1 und aus jeder 1
## eine 0 gemacht
prog_inv = [
    ['z_start','0','1','z_start','R'],
    ['z_start','1','0','z_start','R'],
    ['z_start','_','_','z_ende','L']
]

## Binäres Addieren von 1 (ja ja, ich weiss, das ist eine der Lösungen...
## schauen sie einfach nicht hin und lösen es zuerst selbst)

## Wenn sie richtig fleißig sein wollen, dann arbeiten sie an einem
## Programm, um zwei Binärzahlen, die durch ein Blank getrennt auf dem
## Band stehen, der Kopf steht auf dem ersten Zeichen der ersten Eingabe links,
## zu addieren. Das Ergebnis soll, wieder durch ein oder mehrere Blanks
## getrennt, rechts von den Eingaben stehen. Die Eingaben dürfen sie dabei
## modifizieren, wenn am Ende nur noch das Ergebnis auf dem Band steht, ist
## das auch akzeptal...für's Üben ;)

prog_bin = [
    ['z_start','0','0','z_start','R'], # Nach rechts laufen ohne
    ['z_start','1','1','z_start','R'], # etwas zt verändern
    ['z_start','_','_','z_1','L'], # und wieder eins nach links
    ['z_1','0','1','z_ende','-'],  # Fertig, falls Zeichen unter Lesekopf 0
    ['z_1','1','0','z_1','L'],     # 0 schreiben, falls 1 gefunden, Übertrag
    ['z_1','_','1','z_ende','-']   # Fertig, vorne eine 1 schreiben    
]


def run(name,prog,eingabe,debug=True):
    turing_maschine = Turing(prog,eingabe)

    print("Turingmaschine ",name," ist im Startzustand z_start")
    print("Die Eingabe (mit Eingabezeiger-Position):")
    print("    --> ", turing_maschine.show_input())
    print("    --> ", turing_maschine.show_input_pointer())

    turing_maschine.step(debug)
    print("\n") # Leerzeile ausgeben)
    # input()  # wieder einkommentieren, wenn Sie es nicht im Notebook, sondern direkt in Python ausführen
    # z.B. auf https://repl.it/python3
    

# Jetzt führen wir mal 5 Turingmaschinen aus...

run("Invertiere",prog_inv,['1','0','1','1','0'])
run("Addiere_1",prog_bin,['1','0','1','1','0'])
run("Addiere_1",prog_bin,['1','1','1','1','1'])  # Debug-Ausgabe ausschalten
run("Addiere_1",prog_bin,['_'])
run("Addiere_1",prog_bin,['1','0','2','0'])

# Jetzt können Sie selbst eine Turingmaschine bauen - versuchen Sie sich doch 
# an der Addition von zwei Binärzahlen, s. oben!



Turingmaschine  Invertiere  ist im Startzustand z_start
Die Eingabe (mit Eingabezeiger-Position):
    -->  10110
    -->  ^
0 : -- Execute:  ['z_start', '1', '0', 'z_start', 'R']
    -->  00110
    -->   ^
1 : -- Execute:  ['z_start', '0', '1', 'z_start', 'R']
    -->  01110
    -->    ^
2 : -- Execute:  ['z_start', '1', '0', 'z_start', 'R']
    -->  01010
    -->     ^
3 : -- Execute:  ['z_start', '1', '0', 'z_start', 'R']
    -->  01000
    -->      ^
4 : -- Execute:  ['z_start', '0', '1', 'z_start', 'R']
    -->  01001_____
    -->       ^
5 : -- Execute:  ['z_start', '_', '_', 'z_ende', 'L']
    -->  01001_____
    -->      ^
ERFOLG: Die Maschine hat den Endzustand erreicht!


Turingmaschine  Addiere_1  ist im Startzustand z_start
Die Eingabe (mit Eingabezeiger-Position):
    -->  10110
    -->  ^
0 : -- Execute:  ['z_start', '1', '1', 'z_start', 'R']
    -->  10110
    -->   ^
1 : -- Execute:  ['z_start', '0', '0', 'z_start', 'R']
    -->  10110
    -->    ^
2 : -- Execute:  ['z_s

In [ ]:
# Hier ist Platz für ihre eigenen Experimente!

# Sie können eine Codezelle mit z.B. Shift-Enter ausführen, vorausgesetzt, Sie haben ihren Notebook-Server
# ans Laufen gebracht - installieren Sie dazu Anaconda für Python 3 und starten Sie dann den 
# Anaconde-Navigator und dort wählen Sie "Jupyter Notebook".

# Sie können das Python3-Programm von oben aber auch einfach nach repl.it kopieren und dort ausführen. 
# https://repl.it/python3 - wenn Sie sich da einen Account zulegen, können sie ihre Arbeit sogar
# dort speichern und ihren Kommilitonen (und uns) zugänglich machen!